# Chatbot
by Muhammad Jaysy Ansharulloh

## Import datasets

In [ ]:
entities_filepath = 'datasets/entities.json'
intents_filepath = 'datasets/intents.json'

In [ ]:
import json

In [ ]:
with open(intents_filepath) as fi:
    intents = json.load(fi)

with open(entities_filepath) as fe:
    entities = json.load(fe)

## JSON to dataframe

In [ ]:
import pandas as pd

In [ ]:
df_intents = pd.json_normalize(intents)

In [ ]:
df_intents